In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import ipywidgets as widgets

In [2]:
data_path_sniim = os.path.join("..","..","data","clean_data","precios_sniim.parquet")
data_path_siap = os.path.join("..","..","data","clean_data","siap.parquet.zip")
data_path_siap_wide = os.path.join("..","..","data","clean_data","siap_wide.parquet.zip")

In [3]:
df_precios_sniim = pd.read_parquet(data_path_sniim)
df_cultivos_siap = pd.read_parquet(data_path_siap)
df_cultivos_siap_wide = pd.read_parquet(data_path_siap_wide)

In [4]:
df_precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec
93265,2021-07-13,Pera Angel,Kilogramo,Nayarit,Nayarit,Mercado de abasto 'Adolfo López Mateos' de Tepic,45.00,47.00,46.00
213366,2023-02-09,Chile Guajillo,Kilogramo,Zacatecas,Veracruz,Central de Abasto de Jalapa,115.00,120.00,120.00
1090154,2023-10-02,Cebolla Morada,Arpilla de 25 kg.,Puebla,Tamaulipas,"Módulo de Abasto de Tampico, Madero y Altamira",14.20,18.40,18.40
643903,2021-03-01,Fresa,Kilogramo,Nayarit,Nayarit,Nayarabastos de Tepic,48.00,50.00,49.00
914348,2023-03-07,Chile Caloro,Kilogramo,Sinaloa,Aguascalientes,Centro Comercial Agropecuario de Aguascalientes,18.00,21.00,20.00
1410627,2021-12-01,Acelga,Rollo de 5 kg.,Puebla,Veracruz,Central de Abasto de Minatitlán,12.00,16.00,14.00
957988,2021-09-07,Limon Sin Semilla,Arpilla de 20 kg.,Veracruz,Veracruz,Central de Abasto de Minatitlán,11.00,12.00,12.00
792790,2021-08-18,Chile Anaheim,Kilogramo,Jalisco,Sonora,Central de Abasto de Cd. Obregón,20.00,20.00,20.00
914076,2021-01-26,Cacahuate,Caja de 11 kg.,Nuevo León,Tamaulipas,Módulo de Abasto de Reynosa,43.64,44.55,44.55
1422890,2021-11-08,Chile Pasilla,Kilogramo,Quintana Roo,Quintana Roo,"Mercado de Chetumal, Quintana Roo",200.00,200.00,200.00


In [5]:
df_cultivos_siap_wide.sample(10)

,Entidad,Municipio,Sembrada (ha),Cosechada (ha),Siniestrada (ha),Producción,Rendimiento (udm/ha),Cultivo,fecha
382,Veracruz,Jáltipan,117.00,117.00,0.0,545.22,4.66,chile verde,2022-04-01
65,Chiapas,Huitiupán,2615.00,1315.00,0.0,830.95,0.63,frijol,2023-08-01
231,Hidalgo,San Salvador,71.00,33.00,0.0,1296.90,39.30,avena forrajera en verde,2023-08-01
56,Puebla,Zacapala,10.00,0.00,0.0,0.00,0.00,zanahoria,2020-01-01
212,Jalisco,Yahualica de González Gallo,4653.00,0.00,0.0,0.00,0.00,maíz forrajero en verde,2021-08-01
531,Jalisco,Villa Purificación,584.52,326.00,0.0,2054.74,6.30,maíz grano,2023-06-01
1407,Veracruz,Misantla,2300.00,0.00,0.0,0.00,0.00,maíz grano,2023-04-01
1847,Sonora,San Luis Río Colorado,744.00,0.00,0.0,0.00,0.00,maíz grano,2021-09-01
47,Michoacán,Maravatío,22.00,0.00,0.0,0.00,0.00,zanahoria,2023-05-01
450,Oaxaca,San Juan Colorado,283.00,94.34,0.0,275.15,2.92,maíz grano,2021-03-01


In [6]:
df_precios_sniim["año"] = df_precios_sniim["fecha"].dt.year
df_precios_sniim["mes"] = df_precios_sniim["fecha"].dt.month

In [7]:
def graficar_precios(cultivo):
    subset_df = df_precios_sniim[df_precios_sniim["producto"] == cultivo]
    
    subset_df = subset_df.resample('D', on='fecha').mean(numeric_only=True)
    subset_df["ewm"] = subset_df["precio_frec"].ewm(span=14).mean()
    #subset_df["año"] = subset_df.index.year
    
    fig,ax = plt.subplots()
    
    sns.lineplot(x="fecha",
                 y="ewm",
                 hue="año",
                 data=subset_df,
                 errorbar=None,
                 ax=ax,
                 palette="tab10")
    
    sns.scatterplot(x="fecha",
                    y="precio_frec",
                    hue="año",
                    data=subset_df,
                    ax=ax,
                    palette="tab10",
                    s=25,
                    alpha=0.5)
    
    ax.title.set_text(f"Historial de precios para {cultivo}")
    ax.set_xlabel("Fecha")
    ax.set_ylabel("Precio por kg, MXN")
    ax.grid(axis='y')
    ax.tick_params(axis='x',rotation=90);
    ax.legend(title='Año', labels=["2020","2021","2022","2023"])
    
    fig.set_size_inches(16,9)
    
    plt.show()
    
    return

In [8]:
#graficar_precios("Papaya Maradol")

In [9]:
lista_productos = sorted(list(df_precios_sniim["producto"].unique()))
#lista_productos[:5]

In [10]:
dd = widgets.Dropdown(options=lista_productos,
                      value=lista_productos[0],
                      description="Producto: ")

widgets.interact(graficar_precios,cultivo=dd)

interactive(children=(Dropdown(description='Producto: ', options=('Acelga', 'Aguacate Criollo', 'Aguacate Hass…

<function __main__.graficar_precios(cultivo)>